In [1]:
# Params
input_size = (32,32)
batch_size = 256
num_workers = 5
learning_rate = 1e-06
SGD_momentum = 0.9
epochs = 20

# Training an image classifier

In [2]:
import torch

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


## 1. Loading and normalizing CIFAR10

In [3]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.Resize(input_size), # Must be ahead of transforms.ToTensor()
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print()
print(trainset.__getitem__(0)[0].size(), trainset.__len__())
print(testset.__getitem__(0)[0].size(), testset.__len__())

Files already downloaded and verified
Files already downloaded and verified

torch.Size([3, 32, 32]) 50000
torch.Size([3, 32, 32]) 10000


## 2. Define a GoogLeNet

In [4]:
from googlenet import *

net = GoogLeNet(in_channel=3, num_classes=1, aux_block=True).cuda()

for i in net.named_children():
    print(i)
#param_list = list(net.children())
#print(param_list)

('conv_block1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool1', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('conv_block2', Sequential(
  (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('conv_block3', Sequential(
  (0): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (1): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
))
('maxpool2', MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True))
('inception3a', InceptionBlock(
  (conv_1x1): Sequential(
    (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
    (1): LeakyReLU(negative_slope=0.01, inplace)


## 3. Define a Loss function and optimizer

In [5]:
from torch.optim.lr_scheduler import StepLR

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = torch.nn.CrossEntropyLoss() #error
#criterion = torch.nn.BCELoss() #error
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=SGD_momentum)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=8, gamma=0.96)

## 4. Train the network

In [6]:
times = 1
aux_loss_weight = 0.3

for epoch in range(epochs):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda().float()
        ##print(labels.dtype)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        aux_1, aux_2, outputs = net(inputs) #tuple: (aux1, aux2, input)
        loss1 = criterion(aux_1, labels)
        loss2 = criterion(aux_2, labels)
        loss3 = criterion(outputs, labels)
        
        loss = aux_loss_weight*loss1 + aux_loss_weight*loss2 + loss3
        loss.backward()
        ##scheduler.step() #학습이 안됨.
        optimizer.step()
        
        running_loss += loss.item()
        
    if epoch % times == 0:
        print('[%d] loss: %.3f' %(epoch, running_loss / times))
        running_loss = 0.0
        #torch.save(model,'./data/cifar_model.pkl')

print('\nFinished Training')

[0] loss: 215.211
[1] loss: 204.146
[2] loss: 193.058
[3] loss: 181.938
[4] loss: 170.870
[5] loss: 159.775
[6] loss: 148.696
[7] loss: 137.653
[8] loss: 126.577
[9] loss: 115.449
[10] loss: 104.254
[11] loss: 93.171
[12] loss: 82.119
[13] loss: 70.877
[14] loss: 59.822
[15] loss: 48.600
[16] loss: 37.525
[17] loss: 26.239
[18] loss: 15.046
[19] loss: 3.778

Finished Training


## 5. Test the network on the test data

In [7]:
correct = 0
total = 0

with torch.no_grad():
    net.eval()
    for data in testloader:
        images, labels = data
        images = images.cuda() #.cpu()
        labels = labels.cuda() #.cpu()
        
        outputs = net(images).reshape(batch_size,1)
        #print(labels.size())
        
        _, predicted = torch.max(outputs, dim=1)
        #print(predicted.size())
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %(100 * correct / total))

Accuracy of the network on the 10000 test images: 9 %


In [19]:
correct1 = 0
correct2 = 0
correct3 = 0
total = 0

with torch.no_grad():
    net.train()
    for data in testloader:
        images, labels = data
        images = images.cuda() #.cpu()
        labels = labels.cuda() #.cpu()
        
        aux_1, aux_2, outputs = net(images) #tuple: (aux1, aux2, input)
        aux_1 = aux_1.reshape(batch_size,1)
        aux_2 = aux_2.reshape(batch_size,1)
        outputs = outputs.reshape(batch_size,1)
        #print(outputs)
        #print(aux_1)
        #print(aux_2)
        #print(outputs.size())
        #print(labels.size())
        
        _, predicted1 = torch.max(aux_1, dim=1)
        _, predicted2 = torch.max(aux_2, dim=1)
        _, predicted3 = torch.max(outputs, dim=1)
        #print(predicted.size())
        
        total += labels.size(0)
        correct1 += (predicted1 == labels).sum().item()
        correct2 += (predicted2 == labels).sum().item()
        correct3 += (predicted3 == labels).sum().item()

print('Accuracy_1 of the network on the 10000 test images: %d %%' %(100 * correct1 / total))
print('Accuracy_2 of the network on the 10000 test images: %d %%' %(100 * correct2 / total))
print('Accuracy_3 of the network on the 10000 test images: %d %%' %(100 * correct3 / total))

Accuracy_1 of the network on the 10000 test images: 9 %
Accuracy_2 of the network on the 10000 test images: 9 %
Accuracy_3 of the network on the 10000 test images: 9 %
